In [1]:
%run web_scrape_func_def.ipynb

imports loaded.

> file management func:

    get_content(directory)
    get_last_update(file_path)

> Updating lists func:

    get_participants(event_link)

> fighters_stats_scrape func:

    get_xml(url, headers = {'User-Agent':'Moz...}) # Return web page data in .xml format.
    get_links(url)
    get_fighter_data(url) # Retrive data as list
    get_current_date()

> matches_stats_web_scrape func:

    get_event_date(xml)
    get_event_data(event_link)
    get_later_date(date1, date2)


# Main( )

In [2]:
data_loc = '../data/'

In [3]:
events = pd.read_csv(data_loc + 'update_list_events.csv')
events.tail(3)

,date,event_url
0,2023-04-22,http://ufcstats.com/event-details/15b1b21cd743...
1,2023-04-15,http://ufcstats.com/event-details/3dc3022232b7...


#### Extracting data to data frame:

In [4]:
data = []
column_names = ['result','fighter','opponent',
                'f_kd','o_kd','f_str','o_str','f_td','o_td','f_sub','o_sub',
                'weight_class','method','type','round','time','date','event_url'
               ]

for url in tqdm(events.event_url):
    data.extend(get_event_data(event_link=url))
    
df = pd.DataFrame(data=data, columns=column_names)  
df.tail(3)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.20it/s]


,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
21,win,Loopy Godinez,Cynthia Calvillo,0,0,87,104,0,1,0,0,Women's Strawweight,S-DEC,,3,5:00,08-04-2023,http://ufcstats.com/event-details/3dc3022232b7...
22,win,Ignacio Bahamondes,Trey Ogden,0,0,99,42,0,0,0,0,Catch Weight,U-DEC,,3,5:00,08-04-2023,http://ufcstats.com/event-details/3dc3022232b7...
23,win,Steve Garcia,Shayilan Nuerdanbieke,1,1,17,18,0,1,0,0,Featherweight,KO/TKO,Punch,2,0:36,08-04-2023,http://ufcstats.com/event-details/3dc3022232b7...


#### Merge data with existing file: 

In [5]:
org = pd.read_csv(data_loc + 'ufc_matches_stats.csv')
org = org.fillna('')
#org.date = pd.to_datetime(org['date'], format='%d-%m-%Y')
org.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6451,win,Loopy Godinez,Cynthia Calvillo,0,0,87,104,0,1,0,0,Women's Strawweight,S-DEC,,3,5:00,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...
6452,win,Ignacio Bahamondes,Trey Ogden,0,0,99,42,0,0,0,0,Catch Weight,U-DEC,,3,5:00,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...
6453,win,Steve Garcia,Shayilan Nuerdanbieke,1,1,17,18,0,1,0,0,Featherweight,KO/TKO,Punch,2,0:36,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...


In [6]:
df.date = pd.to_datetime(df['date'], format='%d-%m-%Y')
df = df.fillna('')
data = pd.concat([org, df]).drop_duplicates(subset=['fighter','opponent','date']).reset_index(drop=True)
data.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6475,win,Loopy Godinez,Cynthia Calvillo,0,0,87,104,0,1,0,0,Women's Strawweight,S-DEC,,3,5:00,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...
6476,win,Ignacio Bahamondes,Trey Ogden,0,0,99,42,0,0,0,0,Catch Weight,U-DEC,,3,5:00,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...
6477,win,Steve Garcia,Shayilan Nuerdanbieke,1,1,17,18,0,1,0,0,Featherweight,KO/TKO,Punch,2,0:36,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...


#### Saving file:

In [7]:
data.to_csv(data_loc + 'ufc_matches_stats.csv', index=0)
data.tail(3)

,result,fighter,opponent,f_kd,o_kd,f_str,o_str,f_td,o_td,f_sub,o_sub,weight_class,method,type,round,time,date,event_url
6475,win,Loopy Godinez,Cynthia Calvillo,0,0,87,104,0,1,0,0,Women's Strawweight,S-DEC,,3,5:00,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...
6476,win,Ignacio Bahamondes,Trey Ogden,0,0,99,42,0,0,0,0,Catch Weight,U-DEC,,3,5:00,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...
6477,win,Steve Garcia,Shayilan Nuerdanbieke,1,1,17,18,0,1,0,0,Featherweight,KO/TKO,Punch,2,0:36,2023-04-08 00:00:00,http://ufcstats.com/event-details/3dc3022232b7...


# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>